In [14]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns

from skimage.filters.rank import entropy
from skimage.morphology import rectangle
from skimage.util import img_as_ubyte
from skimage.feature import s
from skimage import shap


from pandas import set_option
set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

filename = 'data/facies_vectors.csv'
training_data = pd.read_csv(filename)
training_data

ImportError: cannot import name 'shape_index'

In [10]:
new_vars = pd.DataFrame()
l_foots = [3,5,10,20]
grouped = training_data.groupby(['Well Name'])

In [11]:
for key in grouped.groups.keys():
    
    depth = grouped.get_group(key)['Depth']
    temp_df = pd.DataFrame()
    temp_df['Depth'] = depth
    for log in ['GR','ILD_log10','DeltaPHI','PHIND','PE']:
      
        temp_data = grouped.get_group(key)[log]
        for l_foot in l_foots:

            footprint = rectangle(l_foot,3)

            image = np.vstack((temp_data,temp_data,temp_data))
            image -= np.median(image) 
            image /= np.max(np.abs(image))
            image = img_as_ubyte(image)

            temp_df[log + '_entropy_foot' + str(l_foot)] = entropy(image,footprint)[0,:]
            
        #The gradient is computed using second order accurate central differences
        #in the interior and either first differences or second order accurate
        #one-sides (forward or backwards) differences at the boundaries.
        
#         temp_df[log + 'gradient1']=np.gradient(temp_data)
#         temp_df[log + 'gradient2']=np.gradient(temp_data,edge_order=2)
            temp_df[log + 'shape_index'] = 
    
    temp_df['Well Name'] = [key for _ in range(len(depth))]
    new_vars = new_vars.append(temp_df)
                

/Users/mablou/anaconda3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/Users/mablou/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:3569: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


In [12]:
new_vars = new_vars.sort_index()

In [13]:
new_vars.to_pickle('data/vars_from_log_entropy.pkl')